In [110]:
import numpy as np
import pandas as pd

# Load ratings Change directory where appropriate
directory = ''
ratings = pd.read_csv(directory+'/ratings.csv', usecols=['userId', 'movieId', 'rating'])
max_userid = ratings['userId'].drop_duplicates().max()+1
max_movieid = ratings['movieId'].drop_duplicates().max() +1
ratings.head()
print(max_userid)
print(max_movieid)

611
193610


In [2]:
# Load movies
movies = pd.read_csv('ml-latest-small/movies.csv', usecols=['movieId', 'title', 'genres'])
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [109]:

from keras.layers import Embedding, Reshape, Concatenate,dot,Input,Dense
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

factors = 100 # The number of dimensional embeddings for movies and users


# Create training set
#shuffled_ratings = ratings.sample(frac=1)

# Shuffling users
#user_ids = shuffled_ratings['userId'].values

# Shuffling movies
#movie_ids = shuffled_ratings['movieId'].values

# Shuffling ratings
#ratings_list = shuffled_ratings['rating'].values


user_ids = ratings['userId'].values
movie_ids = ratings['movieId'].values
ratings_list = ratings['rating'].values


In [ ]:
# user_layers is the embedding layer that creates an User by latent factors matrix.

user_input = Input(shape=(1,), name = 'user_layers')
user_middle  = Embedding(max_userid, factors,input_length=1)(user_input)
user_layers  = Reshape(target_shape=(factors,))(user_middle)


# movie_layers is the embedding layer that creates a Movie by latent factors matrix.
movie_input = Input(shape=(1,), name = 'movie_layers')
movie_middle  = Embedding(max_movieid, factors,input_length=1)(movie_input)
movie_layers  = Reshape(target_shape=(factors,))(movie_middle)

output = dot([user_layers,movie_layers], axes=1)

model = Model(inputs=[user_input,movie_input], 
                  outputs=output)

#model = Model(inputs=[user_layers, movie_layers],output=dot_product)
#model.Dot(left, right)
model.compile(loss='mse', optimizer='adamax')
# Callbacks monitor the validation loss
# Save the model weights each time the validation loss has improved
callbacks = [EarlyStopping('val_loss', patience=2),  
    ModelCheckpoint('movie_weights.h5', save_best_only=True)]
history = model.fit([user_ids,movie_ids], ratings_list, epochs=30, validation_split=.1, verbose=2, callbacks=callbacks)

Train on 90752 samples, validate on 10084 samples
Epoch 1/30
 - 1343s - loss: 13.3690 - val_loss: 11.8135
Epoch 2/30
